<a href="https://colab.research.google.com/github/gurralamanoj/Data-Science-For-Beginners/blob/main/Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
! pip install kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json  
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d zalando-research/fashionmnist
! unzip '/content/fashionmnist.zip'

fashionmnist.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/fashionmnist.zip
replace fashion-mnist_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: fashion-mnist_test.csv  
  inflating: fashion-mnist_train.csv  
  inflating: t10k-images-idx3-ubyte  
  inflating: t10k-labels-idx1-ubyte  
  inflating: train-images-idx3-ubyte  
  inflating: train-labels-idx1-ubyte  


# New Section